In [1]:
#!pip install tqdm
#!pip install pandas
#!pip install selenium

from selenium import webdriver
from tqdm import tqdm
import pandas as pd
import os
import re
import time
import sys

DRIVER_PATH = './chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

url = "https://www.google.com/search?q="

In [ ]:
FOLDER_PATH = "./vocab/"
EXPORT_PATH = "./vocab_w_hotness/"

FILENAMES = os.listdir(FOLDER_PATH)
with tqdm(total=len(FILENAMES), file=sys.stdout) as pbar:
    for file in FILENAMES:
        pbar.set_description('processing: ' + file)
        if file.endswith(".csv"):
            if not os.path.isfile(os.path.join(EXPORT_PATH, file)):
                FILE_PATH = os.path.join(FOLDER_PATH, file)

                df = pd.read_csv(FILE_PATH)

                vocab_hotness_list = []
                for query in df.vocab:
                    while True:
                        try:
                            query_url = url + query
                            driver.get(query_url)

                            result_tag = driver.find_element_by_xpath("//div[@id='result-stats']")
                            result = result_tag.text

                            num = re.findall("[,\d]+", result)
                            num = int(re.sub(",", "", num[0]))
                            #print(f"{query}: {num}")
                            vocab_hotness_list.append((query, num))
                        except:
                            time.sleep(100)
                            continue
                        break

                assert len(df.vocab) == len(vocab_hotness_list)
                vocab_hotness_list = sorted(vocab_hotness_list, key=lambda x: x[1], reverse=True)
                df_new = pd.DataFrame(vocab_hotness_list, columns=["vocab", "hotness"])

                if not os.path.isdir(EXPORT_PATH):
                    os.mkdir(EXPORT_PATH)

                df_new.to_csv(EXPORT_PATH + file, index=False)
            pbar.update(1)
                        
print("Saved all csv in path: " + os.path.abspath(EXPORT_PATH))

processing: buildings_in_the_city.csv:  11%|█▏        | 24/209 [17:27<3:56:36, 76.74s/it]       

In [ ]:
movie theater